In [1]:
import random
import math
from IPython.display import display
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np

In [2]:
class TicTacToe_Game:
    
    def initialise_baord(self) : 
        self.ttt_board = {
                            1: ' ', 2:' ', 3: ' ',
                            4: ' ', 5:' ', 6: ' ',
                            7: ' ', 8:' ', 9: ' '
                         }
    def display_board(self):
        print("\n")
        for row in range(3):
            for col in range(3):
                cell = row * 3 + col + 1
                print(self.ttt_board[cell], end="")
                if col < 2:
                    print(" | ", end="")
            print()
            if row < 2:
                print("---------")
        print()
        
    def tossForFirstMove(self):
        choices = [1,2]
        return random.choice(choices)
        
    def display_board(self):
        print("\n")
        print( self.ttt_board[1], '|', self.ttt_board[2], '|', self.ttt_board[3])
        print(' -+---+-')
        print(self.ttt_board[4], '|', self.ttt_board[5], '|', self.ttt_board[6])
        print(' -+---+-')
        print(self.ttt_board[7], '|', self.ttt_board[8], '|', self.ttt_board[9], "\n")
        
    def validateMove(self, move):
        return self.ttt_board[move] == ' '
        
    def validateDraw(self):
        return all(self.ttt_board[key] != ' ' for key in self.ttt_board.keys())
    
    
    def validateWin(self):
        win_combinations = [
            (1, 2, 3), (4, 5, 6), (7, 8, 9), 
            (1, 4, 7), (2, 5, 8), (3, 6, 9),  
            (1, 5, 9), (7, 5, 3)
        ]

        for combo in win_combinations:
            if (self.ttt_board[combo[0]] == self.ttt_board[combo[1]] == self.ttt_board[combo[2]] != ' '):
                return True

        return False
                        
    def validateWinForLetter(self, mark):
        winning_positions = [
            (1, 2, 3), (4, 5, 6), (7, 8, 9),
            (1, 4, 7), (2, 5, 8), (3, 6, 9),
            (1, 5, 9), (7, 5, 3)
        ]
        for pos in winning_positions:
            if all(self.ttt_board[i] == mark for i in pos):
                return True
        return False

    


In [3]:
class QLearning:
    def __init__(self):
        self.epsilon = 1.0
        self.QLearningStates = {}
  
    getPosition = lambda self, current_board: tuple(tuple(current_board[i+j] for j in range(3)) for i in range(1, 10, 3))

    def getQLearningValue_For_Action(self, current_board, current_position):
        position = self.getPosition(current_board)
        if position not in self.QLearningStates:
            self.QLearningStates[position] = np.zeros((9,))
        return self.QLearningStates[position][current_position - 1]
  

    def getBestPositionFromQLearning(self, current_board, possible_positions):
        return random.choice(possible_positions) if random.random() < self.epsilon else max(possible_positions, key=lambda x: self.getQLearningValue_For_Action(current_board, x))

    def updateQLearningModel(self, current_board, current_position, reward, successive_board, possible_positions):
        bestQValue = max([self.getQLearningValue_For_Action(successive_board, current_position) for next_action in possible_positions], default=0)
        optimisedQVlaue = self.getQLearningValue_For_Action(current_board, current_position) + 0.1 * ((reward + 0.99 * bestQValue) - self.getQLearningValue_For_Action(current_board, current_position))
        position = self.getPosition(current_board)
        self.QLearningStates[position][current_position - 1] = optimisedQVlaue

    def update_epsilon(self):
        self.epsilon = max(self.epsilon * 0.999, 0.1)
        
    def saveQLearningModel(self):
        with open("TicTacToeQLearningModel.pickle", "wb") as file:
            pickle.dump(self.QLearningStates, file)
            
    def loadQLearningModel(self):
        with open("TicTacToeQLearningModel.pickle", "rb") as file:
            self.QLearningStates = pickle.load(file)
            
    def trainQLearningModel(self):
        QLearningWin = SIAgentWin = Draw = 0
        total_episodes = 3000000
        for episode in tqdm(range(total_episodes)):
            ttt_game = TicTacToe_Game()
            ttt_game.initialise_baord()
            current_board = ttt_game.ttt_board

            while True:
                QLearningPossible_Positions = [i for i in range(1, 10) if ttt_game.validateMove(i)]

                if len(QLearningPossible_Positions) == 0:
                    break

                QLearningPosition = self.getBestPositionFromQLearning(current_board, QLearningPossible_Positions)

                if ttt_game.validateMove(QLearningPosition):
                    ttt_game.ttt_board[QLearningPosition] = 'X'

                isQLearningWinner = ttt_game.validateWinForLetter('X')
                isSIAgentWinner = ttt_game.validateWinForLetter('O')
                possibleMoves = [i for i in range(1, 10) if ttt_game.validateMove(i)]

                if isQLearningWinner:
                    QLearningWin += 1
                    self.updateQLearningModel(current_board, QLearningPosition, 1, ttt_game.ttt_board, [])
                    break

                elif isSIAgentWinner:
                    SIAgentWin += 1
                    self.updateQLearningModel(current_board, QLearningPosition, -1, ttt_game.ttt_board, [])
                    break

                elif ttt_game.validateDraw():
                    Draw += 1
                    self.updateQLearningModel(current_board, QLearningPosition, 0, ttt_game.ttt_board, [])
                    break
                else:
                    self.updateQLearningModel(current_board, QLearningPosition, 0, ttt_game.ttt_board, possibleMoves)

                SIAgentPossible_Positions = [i for i in range(1, 10) if ttt_game.validateMove(i)]
                SIAgentPosition = SIAgentPossible_Positions[random.randint(0, len(SIAgentPossible_Positions)-1)]

                if ttt_game.validateMove(SIAgentPosition):
                    ttt_game.ttt_board[SIAgentPosition] = 'O'

                isQLearningWinner = ttt_game.validateWinForLetter('X')
                isSIAgentWinner = ttt_game.validateWinForLetter('O')
                possibleMoves = [i for i in range(1, 10) if ttt_game.validateMove(i)]

                if isQLearningWinner:
                    QLearningWin += 1
                    self.updateQLearningModel(current_board, SIAgentPosition, 1, ttt_game.ttt_board, [])
                    break

                elif isSIAgentWinner:
                    SIAgentWin += 1
                    self.updateQLearningModel(current_board, SIAgentPosition, -1, ttt_game.ttt_board, [])
                    break

                elif ttt_game.validateDraw():
                    Draw += 1
                    self.updateQLearningModel(current_board, SIAgentPosition, 0, ttt_game.ttt_board, [])
                    break
                else:
                    self.updateQLearningModel(current_board, SIAgentPosition, 0, ttt_game.ttt_board, possibleMoves)

                current_board = ttt_game.ttt_board
            self.update_epsilon()

        return QLearningWin, SIAgentWin, Draw, total_episodes

    def play_tic_tac_toe(self, SIAgent_plays_first, ttt_game):
        SI_Agent_Letter = 'O'
        QLearning_Letter = 'X'

        while True:
            if SIAgent_plays_first:
                SIAgentPossible_Positions = [i for i in range(1, 10) if ttt_game.validateMove(i)]

                if len(SIAgentPossible_Positions) == 0:
                    return "Draw"

                SIAgentPosition = SIAgentPossible_Positions[random.randint(0, len(SIAgentPossible_Positions)-1)]

                if ttt_game.validateMove(SIAgentPosition):
                    ttt_game.ttt_board[SIAgentPosition] = SI_Agent_Letter

                if ttt_game.validateWinForLetter(SI_Agent_Letter) : 
                    return "SIAgentWon"

                if ttt_game.validateDraw():
                    return "Draw"

                QLearningPossible_Positions = [i for i in range(1, 10) if ttt_game.validateMove(i)]

                if len(QLearningPossible_Positions) == 0:
                    "Draw"

                QLearningPosition = self.getBestPositionFromQLearning(ttt_game.ttt_board, QLearningPossible_Positions)

                if ttt_game.validateMove(QLearningPosition):
                    ttt_game.ttt_board[QLearningPosition] = QLearning_Letter

                if ttt_game.validateWinForLetter(QLearning_Letter) : 
                    return "QLearningWon"

                if ttt_game.validateDraw():
                    return "Draw"

            else:
                QLearningPossible_Positions = [i for i in range(1, 10) if ttt_game.validateMove(i)]

                if len(QLearningPossible_Positions) == 0:
                    break

                QLearningPosition = self.getBestPositionFromQLearning(ttt_game.ttt_board, QLearningPossible_Positions)

                if ttt_game.validateMove(QLearningPosition):
                    ttt_game.ttt_board[QLearningPosition] = QLearning_Letter

                if ttt_game.validateWinForLetter(QLearning_Letter) : 
                    return "QLearningWon"

                if ttt_game.validateDraw():
                    return "Draw"


                SIAgentPossible_Positions = [i for i in range(1, 10) if ttt_game.validateMove(i)]

                if len(SIAgentPossible_Positions) == 0:
                    return "Draw"

                SIAgentPosition = SIAgentPossible_Positions[random.randint(0, len(SIAgentPossible_Positions)-1)]

                if ttt_game.validateMove(SIAgentPosition):
                    ttt_game.ttt_board[SIAgentPosition] = SI_Agent_Letter

                if ttt_game.validateWinForLetter(SI_Agent_Letter) : 
                    return "SIAgentWon"

                if ttt_game.validateDraw():
                    return "Draw"
                
                    
    

### Training QLearning Model

In [4]:
qLearning = QLearning()

QLearningWin, SIAgentWin, Draw, total_episodes = qLearning.trainQLearningModel()

qLearning.saveQLearningModel()

100%|██████████████████████████████████████████████████████████████████████| 3000000/3000000 [18:54<00:00, 2644.46it/s]


In [5]:

statistics_df = pd.DataFrame(columns=['Game Type', 'Total Number of Games', 'Number of Games Qlearning Won', 'Number of Games Semi-Intelligent player Won', 'Number of Games Drawn'])
statistics_dict = {}
statistics_dict['Game Type'] = 'Training'
statistics_dict['Total Number of Games'] = total_episodes
statistics_dict['Number of Games Qlearning Won'] = QLearningWin
statistics_dict['Number of Games Semi-Intelligent player Won'] = SIAgentWin
statistics_dict['Number of Games Drawn'] = Draw

statistics_df = statistics_df.append(statistics_dict, ignore_index = True)
statistics_df = statistics_df.style.applymap(lambda x:'white-space:nowrap')
display(statistics_df)



,Game Type,Total Number of Games,Number of Games Qlearning Won,Number of Games Semi-Intelligent player Won,Number of Games Drawn
0,Training,3000000,2250425,586352,163223


### First Move: Random

In [6]:
games = 2000
SIAgentWin = QLearningWin = Draw = 0

qLearningPlayer = QLearning() 
qLearningPlayer.loadQLearningModel()

print(f"Current Q Learning model has {len(qLearningPlayer.QLearningStates)} states")

for _ in tqdm(range(games)):
    ttt_game = TicTacToe_Game()
    ttt_game.initialise_baord()
    
    SIAgent_plays_first = False
    if ttt_game.tossForFirstMove() == 1 :
        SIAgent_plays_first = True
    else : 
        SIAgent_plays_first = False
    
    
    
    winner = qLearningPlayer.play_tic_tac_toe(SIAgent_plays_first, ttt_game)

    if winner == 'QLearningWon':
        QLearningWin += 1
    elif winner == 'SIAgentWon':
        SIAgentWin += 1
    else:
        Draw += 1


100%|███████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 16529.34it/s]

Current Q Learning model has 5020 states


In [7]:
statistics_df = pd.DataFrame(columns=['Game Type', 'Total Number of Games', 'Number of Games QLearning Won', 'Number of Games Semi-Intelligent player Won', 'Number of Games Drawn'])
statistics_dict = {}
statistics_dict['Game Type'] = 'First Move: Random'
statistics_dict['Total Number of Games'] = games
statistics_dict['Number of Games QLearning Won'] = QLearningWin
statistics_dict['Number of Games Semi-Intelligent player Won'] = SIAgentWin
statistics_dict['Number of Games Drawn'] = Draw
statistics_df = statistics_df.append(statistics_dict, ignore_index = True)
statistics_df = statistics_df.style.applymap(lambda x:'white-space:nowrap')
display(statistics_df)

,Game Type,Total Number of Games,Number of Games QLearning Won,Number of Games Semi-Intelligent player Won,Number of Games Drawn
0,First Move: Random,2000,869,862,269


### First Move: Always Semi-Intelligent Agent

In [8]:
games = 2000
SIAgentWin = QLearningWin = Draw = 0

qLearningPlayer = QLearning() 
qLearningPlayer.loadQLearningModel()

print(f"Current Q Learning model has {len(qLearningPlayer.QLearningStates)} states")

for _ in tqdm(range(games)):
    ttt_game = TicTacToe_Game()
    ttt_game.initialise_baord()
    
    SIAgent_plays_first = True

    winner = qLearningPlayer.play_tic_tac_toe(SIAgent_plays_first, ttt_game)

    if winner == 'QLearningWon':
        QLearningWin += 1
    elif winner == 'SIAgentWon':
        SIAgentWin += 1
    else:
        Draw += 1


  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]

Current Q Learning model has 5020 states


100%|███████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 17241.57it/s]


In [9]:
statistics_df = pd.DataFrame(columns=['Game Type', 'Total Number of Games', 'Number of Games QLearning Won', 'Number of Games Semi-Intelligent player Won', 'Number of Games Drawn'])
statistics_dict = {}
statistics_dict['Game Type'] = 'First Move: First Move: Semi Intelligent Player'
statistics_dict['Total Number of Games'] = games
statistics_dict['Number of Games QLearning Won'] = QLearningWin
statistics_dict['Number of Games Semi-Intelligent player Won'] = SIAgentWin
statistics_dict['Number of Games Drawn'] = Draw
statistics_df = statistics_df.append(statistics_dict, ignore_index = True)
statistics_df = statistics_df.style.applymap(lambda x:'white-space:nowrap')
display(statistics_df)

,Game Type,Total Number of Games,Number of Games QLearning Won,Number of Games Semi-Intelligent player Won,Number of Games Drawn
0,First Move: First Move: Semi Intelligent Player,2000,599,1128,273


### First Move: Always Q-Learning player

In [10]:
games = 2000
SIAgentWin = QLearningWin = Draw = 0

qLearningPlayer = QLearning() 
qLearningPlayer.loadQLearningModel()

print(f"Current Q Learning model has {len(qLearningPlayer.QLearningStates)} states")

for _ in tqdm(range(games)):
    ttt_game = TicTacToe_Game()
    ttt_game.initialise_baord()
    
    SIAgent_plays_first = False

    winner = qLearningPlayer.play_tic_tac_toe(SIAgent_plays_first, ttt_game)

    if winner == 'QLearningWon':
        QLearningWin += 1
    elif winner == 'SIAgentWon':
        SIAgentWin += 1
    else:
        Draw += 1


  0%|                                                                                         | 0/2000 [00:00<?, ?it/s]

Current Q Learning model has 5020 states


100%|███████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 16949.01it/s]


In [11]:
statistics_df = pd.DataFrame(columns=['Game Type', 'Total Number of Games', 'Number of Games QLearning Won', 'Number of Games Semi-Intelligent player Won', 'Number of Games Drawn'])
statistics_dict = {}
statistics_dict['Game Type'] = 'First Move: Q-Learning Player'
statistics_dict['Total Number of Games'] = games
statistics_dict['Number of Games QLearning Won'] = QLearningWin
statistics_dict['Number of Games Semi-Intelligent player Won'] = SIAgentWin
statistics_dict['Number of Games Drawn'] = Draw
statistics_df = statistics_df.append(statistics_dict, ignore_index = True)
statistics_df = statistics_df.style.applymap(lambda x:'white-space:nowrap')
display(statistics_df)

,Game Type,Total Number of Games,Number of Games QLearning Won,Number of Games Semi-Intelligent player Won,Number of Games Drawn
0,First Move: Q-Learning Player,2000,1183,572,245
